In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from scipy.signal import argrelextrema
from scipy import signal

plt.rcParams['figure.figsize'] = (18, 8)

# Choose Time Interval

In [ ]:
start_date = datetime.date(2017,9,1)
end_date = datetime.date(2018,3,1)

plot = True

# Load Water level data

In [ ]:
# load the water level data
path = "C:/Users/Marleen/Documents/thesis project/Data zaken/Data/Getij data/Waterhoogte Vlieland gemeten per 10 minuten.csv"
df_water_levels = pd.read_csv(path, delimiter=";")

# set date time to pandas timestamp 
df_water_levels['date_time'] = pd.to_datetime(df_water_levels['date_time'])

# drop location and obstype and tijd, datum
df_water_levels = df_water_levels[['date_time','waterheight']]

# select dates 
df_water_levels = df_water_levels.loc[(df_water_levels['date_time'].dt.date >= start_date) & (df_water_levels['date_time'].dt.date < end_date)]

# for 2018, in February one HW and one LW point is missing
if start_date.year == 2017:

    date_time_str = '2018-02-08 14:00:00.00000'  
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    df_water_levels = df_water_levels.append({'date_time' : date_time_obj, 'waterheight' : 43}, ignore_index=True)
    df_water_levels = df_water_levels.sort_values(by='date_time')
df_water_levels.head()

# Load high/low tide time stamps

In [ ]:
# load the data
path = "C:/Users/Marleen/Documents/thesis project/Data zaken/Data/Getij data/Tijden Hoogwater en Laagwater Vlieland vanaf 2016.csv"
df_tide_times = pd.read_csv(path, delimiter=";")

# remove unnessary columns
df_tide_times = df_tide_times[['date_time', 'waterhoogte', 'extreem']]

# set date time to pandas timestamp 
df_tide_times['date_time'] = pd.to_datetime(df_tide_times['date_time'])

# select dates 
df_tide_times = df_tide_times.loc[(df_tide_times['date_time'].dt.date >= start_date) & (df_tide_times['date_time'].dt.date < end_date)]

# transform data from europe time to UTC
df_tide_times['date_time'] = df_tide_times['date_time'].dt.tz_localize('Europe/London').dt.tz_convert('UTC')

# round timestamps to intervals of 10 minutes
df_tide_times['date_time'] = df_tide_times['date_time'].dt.round('10min') 
# df_tide_times['date_time'] = df_tide_times['date_time'].dt.floor('10min') 
# df_tide_times.head()

# Plot some results

In [ ]:
# plt.plot(df_water_levels['date_time'], df_water_levels['waterheight'])
# plt.plot(df_tide_times[df_tide_times['extreem'] == 'HW'].date_time, df_tide_times[df_tide_times['extreem'] == 'HW'].waterhoogte, 'ro')

# Merge data sets and use waterheight from waterlevels data frame

The water levels from both data sets do not correspond perfectly, so choose the water levels in the water levels dataframe to work with.

For winter in 2017/2018 one HW point is missing, but WHY?

In [ ]:
# convert to same date time format as water levels data
df_tide_times['date_time'] = df_tide_times['date_time'].values.astype('datetime64[ns]')

# perform left join on date time
df_water_levels_tides = pd.merge(df_water_levels, df_tide_times[['date_time','extreem']], how='left', on='date_time')

# make sure the data set starts and ends at high water
df_water_levels_tides = df_water_levels_tides.iloc[(df_water_levels_tides['extreem'] == 'HW').values.argmax():]
df_water_levels_tides = df_water_levels_tides.loc[:(df_water_levels_tides[df_water_levels_tides['extreem'] == 'HW']).last_valid_index()]

# for 2018, in February one HW and one LW point is missing
if start_date.year == 2017:

    date_time_str = '2018-02-08 14:00:00.00000'  
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    df_water_levels_tides = df_water_levels_tides.append({'date_time' : date_time_obj, 'waterheight' : 43, 'extreem' : "HW"}, ignore_index=True)
    df_water_levels_tides = df_water_levels_tides.sort_values(by='date_time')
    
    date_time_str = '2018-02-06 19:10:00.00000'  
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    df_water_levels_tides = df_water_levels_tides.append({'date_time' : date_time_obj, 'waterheight' : -126, 'extreem' : "LW"}, ignore_index=True)
    df_water_levels_tides = df_water_levels_tides.sort_values(by='date_time')



# create df with high/low water points 
df_high_water = df_water_levels_tides[df_water_levels_tides['extreem'] == 'HW']
df_low_water = df_water_levels_tides[df_water_levels_tides['extreem'] == 'LW']
  

# create a plot with water levels and high/low tide points
plt.plot(df_water_levels['date_time'], df_water_levels['waterheight'])
plt.plot(df_high_water.date_time, df_high_water.waterheight, 'ro')
plt.plot(df_low_water.date_time, df_low_water.waterheight, 'go')

# Calculate the length of the tidal cycle and the length until low tide

For every tidal cycle we want to know how long the cycle lasts (from high tide to the next high tide) and also the time it takes before low tide is reached (from high tide to low tide).

The time between tidal cycles is calculated as the number of steps per tidal cycle.

(!) Note that if the number of minutes within a time step is changed, this part has to be adapted as well! 

TODO: Hier even toevoegen dat afhankelijk van low en high tide de indices van elkaar af worden getrokken

In [ ]:
# create extra column in df
df_water_levels_tides['time_steps_in_cycle'] = np.nan
df_water_levels_tides['time_steps_to_low_tide'] = np.nan

# get all indices with high water (easy now, see above code block)
high_water_indices = (df_water_levels_tides[df_water_levels_tides['extreem'] == 'HW']).index
low_water_indices =  (df_water_levels_tides[df_water_levels_tides['extreem'] == 'LW']).index

# number of time steps between high tides
num_time_steps_high_tides = np.diff(high_water_indices)

# calculate time steps till low tide 
time_steps_to_low_tide = low_water_indices - high_water_indices[:-1]  
# time_steps_to_low_tide = low_water_indices - high_water_indices

# couple number of time steps to high water time points (exclude the last one, since this point is the end of the simulation)
df_water_levels_tides.time_steps_in_cycle.loc[high_water_indices[:-1]] = num_time_steps_high_tides
# df_water_levels_tides.time_steps_in_cycle.loc[high_water_indices] = num_time_steps_high_tides


# also add number of time steps to low tide for every high tide
df_water_levels_tides.time_steps_to_low_tide.loc[high_water_indices[:-1]] = time_steps_to_low_tide
# df_water_levels_tides.time_steps_to_low_tide.loc[high_water_indices] = time_steps_to_low_tide


# df_water_levels_tides[(df_water_levels_tides['extreem'] == 'HW') | (df_water_levels_tides['extreem'] == 'LW')]
df_water_levels_tides.head()

# Insert the reference weight data

The data has no date time yet, only the day within a year is mentioned. This day should be converted to a date time object and the data set should then be merged with the existing data set (and only on the high tide moments). 

Data comes from Zwart & Hulscher et al. (1996): Seasonal and Annual variation in body weight,...

(Let op! Dit gedeelte moet mogelijk aangepast worden als we andere jaren simuleren en als het een schrikkeljaar is)

In [ ]:
# simulation years
start_year = start_date.year
end_year = end_date.year

# days in new year (this should be 31 + 29 als schrikkeljaar)
days_in_new_y = 31 + 28

# load the data
path = "C:/Users/Marleen/Documents/thesis project/Data zaken/Streefgewicht Scholekster.csv"
df_ref_weight = pd.read_csv(path, delimiter=";")

# add all day numbers (should be 366 in case of schrikkeljaar)
new_df = pd.DataFrame()
new_df['day'] = range(1, 368)
df_ref_weight = pd.merge(new_df, df_ref_weight, how='left')

# interpolate to days
df_ref_weight = df_ref_weight.interpolate(method='linear')

# last row is unnecessary (366 is the first day again)
df_ref_weight = df_ref_weight.iloc[:-1]

# add year to the data (right now for 2017/2018) 
df_ref_weight['year'] = np.where(df_ref_weight['day'] < days_in_new_y + 1, end_year, start_year)

# add date time to dataframe
df_ref_weight['date_time'] = pd.to_datetime(df_ref_weight['year'] * 1000 + df_ref_weight['day'], format='%Y%j')

# merge with rest of the data set
df_water_levels_tides_weight = pd.merge(df_water_levels_tides, df_ref_weight.weight, left_on=[df_water_levels_tides.date_time.dt.year, df_water_levels_tides.date_time.dt.month, df_water_levels_tides.date_time.dt.day],
        right_on=[df_ref_weight.date_time.dt.year, df_ref_weight.date_time.dt.month, df_ref_weight.date_time.dt.day])

# plot
if plot == True:
    plt.plot(df_water_levels_tides_weight.date_time, df_water_levels_tides_weight.weight)
    plt.title('Reference Weight')

# set non HW rows to np.nan (niet zo netjes zo)
df_water_levels_tides_weight.weight.loc[df_water_levels_tides_weight.extreem != "HW"] = np.nan

# remove key columns (kan netter)
df_water_levels_tides_weight = df_water_levels_tides_weight[['date_time', 'waterheight', 'extreem', 'time_steps_in_cycle', 'time_steps_to_low_tide', 'weight']]

# change name for convenience
df_final = df_water_levels_tides_weight

# Insert the temperature data

The temperature data should be coupled to the tidal cycles, ideally we choose the average daily temperature in which most of the cycle is located. Another possibility would be to simply get the temperature at the start of the cycle (implementation would be easier then).

For now: just take the temperature at the start of the cycle, if time is left we can change this. 

In [ ]:
# load the data
path = "C:/Users/Marleen/Documents/thesis project/Data zaken/Data/KNMI data/Weergegevens KNMI Vlieland.csv"
df_temperature = pd.read_csv(path, delimiter=",")

# only keep temperature and day/time columns
df_temperature = df_temperature[['YYYYMMDD', '    T']]

# change column names
df_temperature.columns = ['date', 'temperature']

# set date time to pandas timestamp 
df_temperature['date_time'] = pd.to_datetime(df_temperature['date'], format='%Y%m%d')
df_temperature = df_temperature[['date_time', 'temperature']]

# get part of the data we want
df_temperature = df_temperature[(df_temperature.date_time.dt.date >= start_date) & (df_temperature.date_time.dt.date < end_date)]

# convert to floats
df_temperature.temperature = df_temperature.temperature.astype(float)

# # get mean temperature per day
df_temperature_means = df_temperature.groupby('date_time').mean()
df_temperature_means['date_time'] = df_temperature_means.index 
df_temperature_means = df_temperature_means.reset_index(drop=True)

# change temperature to degrees celcius (instead of 0.1 degrees celcius)
df_temperature_means.temperature = df_temperature_means.temperature / 10

# couple with final df
df_final = pd.merge(df_final, df_temperature_means, 
                    left_on=[df_final.date_time.dt.year, df_final.date_time.dt.month, df_final.date_time.dt.day],
                    right_on=[df_temperature_means.date_time.dt.year, 
                              df_temperature_means.date_time.dt.month, 
                              df_temperature_means.date_time.dt.day])

# grab final columns (keys are in there now)
df_final = df_final[['date_time_x', 'waterheight', 'extreem', 'time_steps_in_cycle', 
                     'time_steps_to_low_tide', 'weight', 'temperature']]
df_final=df_final.rename(columns = {'date_time_x':'date_time'})

# plot to check if data is logical
if plot == True:
    plt.plot(df_final.date_time, df_final.temperature)

# set temperature of non HW rows to zero
df_final.temperature.loc[df_final.extreem != "HW"] = np.nan
df_final.head()